In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from scipy.fft import fft, fftfreq
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm.auto import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

# הגדרת matplotlib לעברית
plt.rcParams['font.family'] = ['Arial Unicode MS', 'Tahoma', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False


DATA_PATH = "data/"


def load_and_prepare_data(data_path_folder):
    file_mapping = {
        'car_nothing.csv': 'quiet',
        'carnew.csv': 'vehicle',
        'human_nothing.csv': 'quiet',
        'human.csv': 'human'
    }
    label_encoding = {'quiet': 0, 'vehicle': 1, 'human': 2}
    all_data = []
    all_labels = []
    print("Starting data loading...")
    if not os.path.exists(data_path_folder):
        print(f"Data folder {data_path_folder} not found. Please create it and add data files.")
        return np.array([]), np.array([])
    for filename, activity_type in file_mapping.items():
        filepath = os.path.join(data_path_folder, filename)
        if not os.path.exists(filepath):
            print(f"Error: File not found at {filepath}. Skipping.")
            continue
        try:
            df = pd.read_csv(filepath, header=None)
            if not df.empty and df.shape[1] > 0:
                data = df.iloc[:, 0].values
                label_code = label_encoding[activity_type]
                all_data.extend(data)
                all_labels.extend([label_code] * len(data))
            else:
                print(f"Warning: File {filename} is empty or has no data columns. Skipping.")
        except Exception as e:
            print(f"Error reading {filename}: {e}")
    all_data_np = np.array(all_data)
    all_labels_np = np.array(all_labels)
    if len(all_data_np) > 0:
        print(f"Total data points loaded: {len(all_data_np)}")
    else:
        print("No data was loaded.")
    return all_data_np, all_labels_np

C:\Users\gil70\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def prepare_raw_data(data_dict, labels_dict, window_size_seconds=2, sample_rate=1000):
    """הכנת נתונים גולמיים עם חלונות זמן"""
    
    window_size = window_size_seconds * sample_rate  # 2000 דגימות
    
    raw_windows = []
    labels = []
    
    # חישוב מספר כולל של חלונות לtqdm
    total_windows = 0
    for filename, signal_data in data_dict.items():
        total_windows += (signal_data.shape[0] - window_size) // (window_size // 2)
    
    print(f"🔄 יצירת {total_windows} חלונות של {window_size_seconds} שניות...")
    
    with tqdm(total=total_windows, desc="עיבוד חלונות גולמיים") as pbar:
        for filename, signal_data in data_dict.items():
            label = labels_dict[filename]
            
            # חלונות עם overlap של 50%
            step_size = window_size // 2
            
            for start_idx in range(0, signal_data.shape[0] - window_size + 1, step_size):
                end_idx = start_idx + window_size
                window = signal_data[start_idx:end_idx]
                
                if window.shape[0] == window_size:
                    raw_windows.append(window)
                    labels.append(label)
                
                pbar.update(1)
                if pbar.n >= total_windows:
                    break
    
    return np.array(raw_windows), np.array(labels)


In [4]:
# 3. מודל CNN פשוט לנתונים גולמיים
def create_raw_cnn_model(input_shape, num_classes):
    """מודל CNN לנתונים גולמיים (2D: זמן × חיישנים)"""
    
    model = keras.Sequential([
        # Input layer
        layers.Input(shape=input_shape),
        
        # הוספת dimension channel
        layers.Reshape(input_shape + (1,)),
        
        # CNN layers - רואה דפוסים בזמן ובין חיישנים
        layers.Conv2D(16, (5, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 1)),
        layers.BatchNormalization(),
        
        layers.Conv2D(32, (5, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 1)),
        layers.BatchNormalization(),
        
        layers.Conv2D(64, (3, 2), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 1)),
        layers.BatchNormalization(),
        
        layers.Conv2D(128, (3, 2), activation='relu', padding='same'),
        layers.GlobalAveragePooling2D(),
        
        # Dense layers
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        
        # Output
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model